In [18]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import label_binarize
import gc

In [64]:
gc.collect()

254

## 读入数据

### train1数据

In [2]:
path4 = 'E:\\MySQL_data\\repeat_buyers_prediction\\data_format1\\train_format1.csv'

In [3]:
df_train1 = pd.read_csv(path4)

In [4]:
df_train1.head()

,user_id,merchant_id,label
0,34176,3906,0
1,34176,121,0
2,34176,4356,1
3,34176,2217,0
4,230784,4818,0


In [5]:
df_train1[df_train1.label.isnull()]

,user_id,merchant_id,label


### user_info数据

In [6]:
'''
path5 = 'E:\\MySQL_data\\repeat_buyers_prediction\\data_format1\\user_info_format1.csv'

userinfo = pd.read_csv(path5)

# age_gender、gender中有空值存在，故需要处理
userinfo.fillna( value={'gender':2.0}, inplace=True )
userinfo.fillna( value={'age_range':0.0}, inplace=True)

# 在age_range中8、7代表的是同一年龄组，所以需要处理
userinfo.replace( to_replace={'age_range':8.0}, value=7.0, inplace=True )

userinfo.to_csv( 'E:\\MySQL_data\\repeat_buyers_prediction\\data_format1\\user_info_format1_handled.csv', 
                index=False,encoding='utf-8' )
# 以后再读取user_info文件时就直接读取这个已经初步处理了的
'''

In [18]:
path5 = 'E:\\MySQL_data\\repeat_buyers_prediction\\data_format1\\user_info_format1_handled.csv'
userinfo = pd.read_csv(path5)

In [19]:
userinfo.head()

,user_id,age_range,gender
0,376517,6.0,1.0
1,234512,5.0,0.0
2,344532,5.0,0.0
3,186135,5.0,0.0
4,30230,5.0,0.0


### 1.7G user_log数据

In [3]:
path6 = 'E:\\MySQL_data\\repeat_buyers_prediction\\data_format1\\user_log_format1.csv'   

In [4]:
userlog = pd.read_csv(path6)

In [22]:
userlog.head()

,user_id,item_id,cat_id,seller_id,brand_id,time_stamp,action_type
0,328862,323294,833,2882,2661.0,829,0
1,328862,844400,1271,2882,2661.0,829,0
2,328862,575153,1271,2882,2661.0,829,0
3,328862,996875,1271,2882,2661.0,829,0
4,328862,1086186,1271,1253,1049.0,829,0


## 处理user_info数据

In [24]:
userinfo1 = pd.get_dummies( userinfo['age_range'], prefix='age' )

In [26]:
userinfo2 = pd.get_dummies( userinfo['gender'], prefix='gender' )

In [28]:
u1 = userinfo[['user_id']].join(userinfo1).join(userinfo2)

In [29]:
u1.head()

,user_id,age_0.0,age_1.0,age_2.0,age_3.0,age_4.0,age_5.0,age_6.0,age_7.0,gender_0.0,gender_1.0,gender_2.0
0,376517,0,0,0,0,0,0,1,0,0,1,0
1,234512,0,0,0,0,0,1,0,0,1,0,0
2,344532,0,0,0,0,0,1,0,0,1,0,0
3,186135,0,0,0,0,0,1,0,0,1,0,0
4,30230,0,0,0,0,0,1,0,0,1,0,0


## 处理user_log数据

### 建立user_id_item_count
每个用户操作过的商品数量

In [ ]:
u2 = userlog[['user_id', 'item_id']].groupby(['user_id']).count()

In [ ]:
u2.rename( columns={'item_id': 'user_item_count'}, inplace=True )

In [37]:
u2.reset_index('user_id', inplace=True )

In [38]:
u2.head()

,user_id,user_item_count
0,1,33
1,2,63
2,3,68
3,4,50
4,5,173


### 建立user_id_item_action_type_count
每个用户对每种商品的操作方式的次数

In [33]:
u3 = userlog[['user_id', 'item_id', 'action_type']].groupby(['user_id', 'item_id', 'action_type']).size()  

In [34]:
u4 = u3.unstack(level='action_type').fillna(0.0)

In [42]:
u5 = u4.reindex( columns=pd.Index([0, 1, 2, 3], name=None ), copy=False )

In [43]:
u5.rename( columns={i:'user_item_aciton_type{}'.format(i) for i in range(4) }, 
           inplace = True)

In [44]:
u6 = u5.reset_index() 

In [45]:
u6.head()

,user_id,item_id,user_item_aciton_type0,user_item_aciton_type1,user_item_aciton_type2,user_item_aciton_type3
0,1,112203,4.0,0.0,1.0,0.0
1,1,181459,1.0,0.0,0.0,0.0
2,1,411984,1.0,0.0,0.0,0.0
3,1,452339,1.0,0.0,0.0,0.0
4,1,452837,1.0,0.0,0.0,0.0


### 建立user_seller_time_stamp
每个用户在不同商家里逛街的次数（同一天算一次）

In [ ]:
u7 = userlog[['user_id', 'seller_id', 'time_stamp']].copy()

In [47]:
u7.drop_duplicates(inplace=True) # 去重方可实现每天算一次

In [48]:
u8 = u7.groupby(['user_id', 'seller_id']).count()

In [50]:
u9 = u8.reset_index()

In [53]:
u9.sample(8)

,user_id,seller_id,time_stamp
8552106,258255,2860,1
10325997,311326,2440,1
12180371,366989,533,1
11311041,340683,3116,4
3685114,111349,2086,1
7198985,217467,1461,5
11741864,353730,2401,1
3622620,109433,2527,3


### user_seller_action_type
用户在不同seller中不同操作的次数

In [54]:
u10 = userlog[['user_id', 'seller_id', 'action_type']].copy()

In [55]:
lab = label_binarize(u10[['action_type']], classes=[0, 1, 2, 3])

In [ ]:
u11 = u10[['user_id', 'seller_id']].join(pd.DataFrame(lab,
                                                      columns=pd.Index(['user_seller_aciton_type{}'.format(i) for i in range(4)]) ) )  

到这里的时候内存就不够了，所以放弃用pandas继续本项目